# 価格は正しいです

今日、商品の価格を推定するためのより複雑なソリューションを構築しています。

1。Day2.0ノートブック：400,000のトレーニングデータを使用してRAGデータベースを作成する
2。Day2.1ノートブック：2dで視覚化します
3。2.2日目ノートブック：3Dで視覚化します
4。Day2.3ノートブック：GPT-4O-MINIでRAGパイプラインを構築してテストする
5。Day2.4ノートブック：（a）ランダムフォレストプライザー（b）すべての価格からの寄付を可能にするアンサンブルプライザーを作成する

うーん！それは一日で乗り越えるのにたくさんのことです！

＃＃ ご注意ください：

私たちはすでに、独自のファインチューニングされたLLMを使用して非常に強力な製品推定器を持っています。ほとんどの人はそれに非常に満足するでしょう！これらの追加の手順を追加する主な理由は、RAGとエージェントワークフローで専門知識を深めることです。

##今日は速く行きます！あなたの帽子を握ってください。

In [ ]:
# 輸入

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from testing import Tester

In [ ]:
# 環境

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Huggingfaceにログインします

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# 顔を抱き締めるためにログインした後の別の輸入品 - ありがとう、Trung N.！

from items import Item

In [ ]:
openai = OpenAI()

In [ ]:
# テストピクルスファイルにロードします
# 2.0 Notebookの「PKLファイルに戻る」セクションを参照してください
# このtest.pklファイルを取得する手順について

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [ ]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [ ]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [ ]:
DB = "products_vectorstore"

In [ ]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [ ]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [ ]:
description(test[0])

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def vector(item):
    return model.encode([description(item)])

In [ ]:
def find_similars(item):
    results = collection.query(query_embeddings=vector(item).astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [ ]:
print(test[1].prompt)

In [ ]:
documents, prices = find_similars(test[1])

In [ ]:
print(make_context(documents, prices))

In [ ]:
print(messages_for(test[1], documents, prices))

In [ ]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
get_price("The price for this is $99.99")

In [ ]:
# GPT-4O-MINIの関数

def gpt_4o_mini_rag(item):
    documents, prices = find_similars(item)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages_for(item, documents, prices),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
gpt_4o_mini_rag(test[1])

In [ ]:
test[1].price

In [ ]:
Tester.test(gpt_4o_mini_rag, test)

## オプションの追加：Openaiの代わりにDeepSeek API呼び出しを試す

DeepSeek APIキーがある場合は、ここでは代替実装として使用します。それ以外の場合は、次のセクションにスキップします。

In [ ]:
# OpenAIクライアントPythonライブラリを使用してDeepSeekに接続します

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_via_openai_client = OpenAI(api_key=deepseek_api_key,base_url="https://api.deepseek.com")

In [ ]:
# DeepSeekは非常に過剰にサブスクライブされ、時には失敗するため、ここにいくつかの再試行ロジックを追加しました。

def deepseek_api_rag(item):
    documents, prices = find_similars(item)
    retries = 8
    done = False
    while not done and retries > 0:
        try:
            response = deepseek_via_openai_client.chat.completions.create(
                model="deepseek-chat", 
                messages=messages_for(item, documents, prices),
                seed=42,
                max_tokens=8
            )
            reply = response.choices[0].message.content
            done = True
        except Exception as e:
            print(f"Error: {e}")
            retries -= 1
    return get_price(reply)

In [ ]:
deepseek_api_rag(test[1])

In [ ]:
Tester.test(deepseek_api_rag, test)

## そして今、これを「エージェント」クラスで包みます

In [ ]:
from agents.frontier_agent import FrontierAgent

In [ ]:
# 何が起こっているのかを見ることができるように、ログを印刷しましょう

import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
agent = FrontierAgent(collection)

In [ ]:
agent.price("Quadcast HyperX condenser mic for high quality podcasting")

In [ ]:
from agents.specialist_agent import SpecialistAgent

In [ ]:
agent2 = SpecialistAgent()

In [ ]:
agent2.price("Quadcast HyperX condenser mic for high quality podcasting")